In [2]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("Book1.csv")  # Update with your file path

# Convert from wide to long format
df_long = df.melt(id_vars=["Year", "Month"], var_name="Day", value_name="Wind Speed")

# Convert Day to integer
df_long["Day"] = df_long["Day"].astype(int)

# Drop rows with missing Wind Speed (e.g. 31 Feb, 30 Apr etc.)
df_long = df_long.dropna(subset=["Wind Speed"])

# Create proper Date column in DD/MM/YYYY format
df_long["Date"] = df_long["Day"].astype(str).str.zfill(2) + "/" + \
                  df_long["Month"].astype(str).str.zfill(2) + "/" + \
                  df_long["Year"].astype(str)

# Final output
df_final = df_long[["Date", "Wind Speed"]].reset_index(drop=True)

# Save to file (optional)
df_final.to_csv("Wind Speed_Daily_Distribution.csv", index=False)

# Show preview
print(df_final.head(15))


          Date Wind Speed
0   01/01/1981          2
1   01/02/1981          2
2   01/03/1981        7.9
3   01/04/1981        2.7
4   01/05/1981          2
5   01/06/1981          2
6   01/07/1981        5.2
7   01/08/1981          2
8   01/09/1981        5.8
9   01/10/1981        2.9
10  01/11/1981          0
11  01/12/1981          0
12  01/01/1982          3
13  01/02/1982          3
14  01/03/1982          0


In [3]:
import pandas as pd

df = pd.read_csv("Wind Speed_Daily_Distribution.csv")


In [4]:
print(df.head(10))


         Date Wind Speed
0  01/01/1981          2
1  01/02/1981          2
2  01/03/1981        7.9
3  01/04/1981        2.7
4  01/05/1981          2
5  01/06/1981          2
6  01/07/1981        5.2
7  01/08/1981          2
8  01/09/1981        5.8
9  01/10/1981        2.9


In [5]:
df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y")
df = df.sort_values("Date").reset_index(drop=True)
df["Date"] = df["Date"].dt.strftime("%d/%m/%Y")


ValueError: day is out of range for month, at position 4705. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [6]:
# Step 1: Try parsing the date safely
df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y", errors='coerce')

# Step 2: Find invalid dates (NaT)
invalid_dates = df[df["Date"].isna()]
print("Invalid dates found at positions:", invalid_dates.index.tolist())

# Step 3: Drop or fix invalid dates
df = df.dropna(subset=["Date"])  # drops rows with invalid dates

# Step 4: Sort and format
df = df.sort_values("Date").reset_index(drop=True)
df["Date"] = df["Date"].dt.strftime("%d/%m/%Y")


Invalid dates found at positions: [4705, 4717, 4729, 4753, 4765, 4777, 4801, 4813, 4825, 4849, 4861]


In [7]:
df

,Date,Wind Speed
0,01/01/1981,2
1,02/01/1981,2.5
2,03/01/1981,1.0
3,04/01/1981,1.0
4,05/01/1981,2.5
...,...,...
5108,27/12/1994,5.0
5109,28/12/1994,3.0
5110,29/12/1994,2.0
5111,30/12/1994,3.0


In [8]:
df.to_csv("Wind Speed_Daily_Distribution_Fixed.csv", index=False)


In [9]:
# Check for missing values
missing_summary = df.isnull().sum()

# Show where missing values exist
print(missing_summary)

Date          0
Wind Speed    0
dtype: int64


In [10]:
# Check if any cell contains '***'
contains_stars = df.applymap(lambda x: '****' in str(x)).any().any()

if contains_stars:
    print("CSV file er kono cell e '****' ache.")
else:
    print("CSV file er kono cell e '****' nei.")

CSV file er kono cell e '****' ache.


C:\Users\Hp\AppData\Local\Temp\ipykernel_11136\525864041.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  contains_stars = df.applymap(lambda x: '****' in str(x)).any().any()


In [11]:
df

,Date,Wind Speed
0,01/01/1981,2
1,02/01/1981,2.5
2,03/01/1981,1.0
3,04/01/1981,1.0
4,05/01/1981,2.5
...,...,...
5108,27/12/1994,5.0
5109,28/12/1994,3.0
5110,29/12/1994,2.0
5111,30/12/1994,3.0


In [12]:
# Je cells e '***' ache segular index list ber korbo
cells_with_stars = []

for row_idx, row in df.iterrows():
    for col in df.columns:
        cell_value = str(row[col])
        if '***' in cell_value:
            cells_with_stars.append((row_idx, col))

if cells_with_stars:
    print("Cells with '***' found at:")
    for cell in cells_with_stars:
        print(f"Row: {cell[0]}, Column: {cell[1]}")
else:
    print("No cells with '***' found.")

Cells with '***' found at:
Row: 97, Column: Wind Speed
Row: 1216, Column: Wind Speed
Row: 1217, Column: Wind Speed
Row: 2356, Column: Wind Speed
Row: 2357, Column: Wind Speed
Row: 2358, Column: Wind Speed
Row: 2359, Column: Wind Speed
Row: 2360, Column: Wind Speed
Row: 2361, Column: Wind Speed
Row: 2362, Column: Wind Speed


In [13]:
# 'Wind Speed' column e '***' ke NaN replace koro
df['Wind Speed'] = df['Wind Speed'].replace('***', pd.NA)

# 'Wind Speed' column ke numeric convert koro (NaN thakbe jekhane '*** chilo)
df['Wind Speed'] = pd.to_numeric(df['Wind Speed'], errors='coerce')

# Mean calculate koro (NaN ignore kore)
mean_val = df['Wind Speed'].mean()

# NaN gula ke mean diye replace koro
df['Wind Speed'].fillna(mean_val, inplace=True)

C:\Users\Hp\AppData\Local\Temp\ipykernel_11136\3309269723.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Wind Speed'].fillna(mean_val, inplace=True)


In [14]:
df

,Date,Wind Speed
0,01/01/1981,2.0
1,02/01/1981,2.5
2,03/01/1981,1.0
3,04/01/1981,1.0
4,05/01/1981,2.5
...,...,...
5108,27/12/1994,5.0
5109,28/12/1994,3.0
5110,29/12/1994,2.0
5111,30/12/1994,3.0


In [15]:
# Je cells e '***' ache segular index list ber korbo
cells_with_stars = []

for row_idx, row in df.iterrows():
    for col in df.columns:
        cell_value = str(row[col])
        if '***' in cell_value:
            cells_with_stars.append((row_idx, col))

if cells_with_stars:
    print("Cells with '***' found at:")
    for cell in cells_with_stars:
        print(f"Row: {cell[0]}, Column: {cell[1]}")
else:
    print("No cells with '***' found.")

No cells with '***' found.


In [16]:
# Replace sesh porjonto file save korte paro
df.to_csv("Wind Speed.csv", index=False)

In [17]:
df

,Date,Wind Speed
0,01/01/1981,2.0
1,02/01/1981,2.5
2,03/01/1981,1.0
3,04/01/1981,1.0
4,05/01/1981,2.5
...,...,...
5108,27/12/1994,5.0
5109,28/12/1994,3.0
5110,29/12/1994,2.0
5111,30/12/1994,3.0
